<pre>
   _____                            _                __      ___     _                  _                        _ 
  / ____|                          | |               \ \    / (_)   (_)                | |                      | |
 | |     ___  _ __ ___  _ __  _   _| |_ ___ _ __      \ \  / / _ ___ _  ___  _ __      | |__   __ _ ___  ___  __| |
 | |    / _ \| '_ ` _ \| '_ \| | | | __/ _ \ '__|      \ \/ / | / __| |/ _ \| '_ \     | '_ \ / _` / __|/ _ \/ _` |
 | |___| (_) | | | | | | |_) | |_| | ||  __/ |          \  /  | \__ \ | (_) | | | |    | |_) | (_| \__ \  __/ (_| |
  \_____\___/|_| |_| |_| .__/ \__,_|\__\___|_|           \/   |_|___/_|\___/|_| |_|    |_.__/ \__,_|___/\___|\__,_|
  _______              | | __  __                                                    _                             
 |__   __|             |_||  \/  |                                                  | |                            
    | |_ __ ___  ___      | \  / | ___  __ _ ___ _   _ _ __ ___ _ __ ___   ___ _ __ | |_                           
    | | '__/ _ \/ _ \     | |\/| |/ _ \/ _` / __| | | | '__/ _ \ '_ ` _ \ / _ \ '_ \| __|                          
    | | | |  __/  __/     | |  | |  __/ (_| \__ \ |_| | | |  __/ | | | | |  __/ | | | |_                           
    |_|_|  \___|\___|     |_|  |_|\___|\__,_|___/\__,_|_|  \___|_| |_| |_|\___|_| |_|\__|                          
                                                                                                                   
                                                                                                                   

## Preliminary

### imports

In [1]:
# imports

import os
import cv2
from glob import glob
import numpy as np

### frame extraction

In [4]:
# frame extraction from video

def extract_frames(video_path, output_folder = "./assets/extracted", capture_every_frame=30):
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    os.makedirs(f"{output_folder}/{video_name}", exist_ok=True)

    video = cv2.VideoCapture(video_path)

    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in {video_name}: {total_frames}")

    frames = 0

    while total_frames > frames*capture_every_frame:
        video.set(cv2.CAP_PROP_POS_FRAMES, frames * capture_every_frame)
        success, image = video.read()

        # sometimes this fails, corrupt frames in vide? idk
        if success:
            cv2.imwrite(f"./{output_folder}/{video_name}/{video_name}_{frames}.png", image)

        frames += 1

    video.release()

In [5]:
calibration_video = "./assets/original/calibration.MP4"
eastbound_video   = "./assets/original/eastbound_20240319.MP4"
westbound_video   = "./assets/original/westbound_20240319.MP4"


extract_frames(calibration_video)
extract_frames(eastbound_video)
extract_frames(westbound_video)

Total frames in calibration: 2995
Total frames in eastbound_20240319: 18786
Total frames in westbound_20240319: 12371


### calibration

In [6]:
def calibrate_camera(calibration_images):
    objp = np.zeros((6*8, 3), np.float32)
    objp[:, :2] = np.mgrid[0:8, 0:6].T.reshape(-1, 2)

    objpoints = []
    imgpoints = []


    for img_path in calibration_images:
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        ret, corners = cv2.findChessboardCorners(gray, (8, 6), None)

        if ret:
            objpoints.append(objp)
            imgpoints.append(corners)


    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    return ret, mtx, dist, rvecs, tvecs

In [7]:
calibration_images = glob('./assets/extracted/calibration/*.png')

ret, mtx, dist, rvecs, tvecs = calibrate_camera(calibration_images)

### Undestortion

In [8]:
def undistort_images(images, output_loc="./assets/undestorted"):
    os.makedirs(f"{output_loc}", exist_ok=True)

    for img_path in images:
        img = cv2.imread(img_path)
        frame_name = os.path.splitext(os.path.basename(img_path))[0]
        h,  w = img.shape[:2]
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))


        undistorted_image = cv2.undistort(img, mtx, dist, None, mtx)
        cv2.imwrite(f"./{output_loc}/{frame_name}.png", undistorted_image)

        


In [9]:
westbound_images   = glob('./assets/extracted/westbound_20240319/*.png')
eastbound_images   = glob('./assets/extracted/eastbound_20240319/*.png')

undistort_images(westbound_images, output_loc="./assets/undestorted/westbound")
undistort_images(eastbound_images, output_loc="./assets/undestorted/eastbound")

## Tree Detection

In [ ]:
# https://universe.roboflow.com/trees-sam/trees-detection-yac52/model/2
# https://github.com/norlab-ulaval/PercepTreeV1

## Tree Triangulation and mapping

## Tree Measurement

In [6]:
# #